In [111]:
from tensorflow.keras.models import load_model
import pickle
import pandas as pd

In [112]:
model = load_model('model.h5')

with open('gender_encode.pkl','rb') as file:
    gender_encode = pickle.load(file)
with open('onehot_encode_geo.pkl','rb') as file:
    onehot_encode_geo = pickle.load(file)
with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [113]:
input_data = {
    'CreditScore' : 650,
    'Geography' : 'France',
    'Gender' : 'Male',
    'Age' :  22,
    'Tenure' : 6,
    'Balance' : 22000,
    'NumOfProducts' : 2,
    'HasCrCard' : 1,
    'IsActiveMember' : 1, 
    'EstimatedSalary' : 34000
}
input_data = pd.DataFrame([input_data])

In [114]:
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,650,France,Male,22,6,22000,2,1,1,34000


In [115]:
input_data["Gender"] = gender_encode.transform(input_data["Gender"])


In [116]:
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,650,France,1,22,6,22000,2,1,1,34000


In [117]:
geo_encoded = onehot_encode_geo.transform([input_data["Geography"]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=onehot_encode_geo.get_feature_names_out(['Geography']))

c:\Users\xyzai\PYTHON_DL\deeplearn\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [118]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [119]:
input_data = pd.concat([input_data.drop("Geography",axis=1),geo_encoded_df],axis=1)

In [120]:
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,650,1,22,6,22000,2,1,1,34000,1.0,0.0,0.0


In [121]:
scaled_data = scaler.transform(input_data)

In [122]:
scaled_data

array([[-0.01958202,  0.87388124, -1.58882889,  0.34778223, -0.86443667,
         0.82444306,  0.62670381,  0.94738229, -1.18088021,  0.99203175,
        -0.55732934, -0.59120865]])

In [123]:
prediction = model.predict(scaled_data)
prediction

1/1 [==============================] - 0s 39ms/step


array([[0.00018621]], dtype=float32)

In [124]:
prediction_proba = prediction[0][0]
prediction_proba

0.00018621262

In [125]:
if prediction_proba > 0.5:
    print("likely to churn")
else:
    print("not likely to churn")

not likely to churn
